In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import zipfile
zip_path = "/content/drive/MyDrive/ML/data.zip"
unzip_path = "/content/data"
if not os.path.exists(unzip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(unzip_path)

In [3]:
import warnings

import numpy as np
import pandas as pd

import cv2
from PIL import Image
from matplotlib import pyplot as plt
import os
import sys
import warnings
import timm

import torch
import torch.nn as nn
from sklearn.metrics import precision_score, f1_score, recall_score, confusion_matrix

import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

import torchvision
import torchvision.transforms as transforms
from torchvision import datasets

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
warnings.filterwarnings("ignore")

In [5]:
class VggFeatures(nn.Module):
    def __init__(self, drop=0.2):
        super().__init__()

        self.conv1a = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, padding=1)
        self.conv1b = nn.Conv2d(64, out_channels=64, kernel_size=3, padding=1)

        self.conv2a = nn.Conv2d(64, 128, 3, padding=1)
        self.conv2b = nn.Conv2d(128, 128, 3, padding=1)

        self.conv3a = nn.Conv2d(128, 256, 3, padding=1)
        self.conv3b = nn.Conv2d(256, 256, 3, padding=1)

        self.conv4a = nn.Conv2d(256, 512, 3, padding=1)
        self.conv4b = nn.Conv2d(512, 512, 3, padding=1)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.bn1a = nn.BatchNorm2d(64)
        self.bn1b = nn.BatchNorm2d(64)

        self.bn2a = nn.BatchNorm2d(128)
        self.bn2b = nn.BatchNorm2d(128)

        self.bn3a = nn.BatchNorm2d(256)
        self.bn3b = nn.BatchNorm2d(256)

        self.bn4a = nn.BatchNorm2d(512)
        self.bn4b = nn.BatchNorm2d(512)

        self.lin1 = nn.Linear(512 * 2 * 2, 4096)
        self.lin2 = nn.Linear(4096, 4096)

        self.drop = nn.Dropout(p=drop)

    def forward(self, x):
        x = F.relu(self.bn1a(self.conv1a(x)))
        x = F.relu(self.bn1b(self.conv1b(x)))
        x = self.pool(x)

        x = F.relu(self.bn2a(self.conv2a(x)))
        x = F.relu(self.bn2b(self.conv2b(x)))
        x = self.pool(x)

        x = F.relu(self.bn3a(self.conv3a(x)))
        x = F.relu(self.bn3b(self.conv3b(x)))
        x = self.pool(x)

        x = F.relu(self.bn4a(self.conv4a(x)))
        x = F.relu(self.bn4b(self.conv4b(x)))
        x = self.pool(x)
        # print(x.shape)

        x = x.view(-1, 512 * 2 * 2)
        x = F.relu(self.drop(self.lin1(x)))
        x = F.relu(self.drop(self.lin2(x)))

        return x


class Vgg(VggFeatures):
    def __init__(self, drop=0.2):
        super().__init__(drop)
        self.lin3 = nn.Linear(4096, 7)

    def forward(self, x):
        x = super().forward(x)
        x = self.lin3(x)
        return x

In [6]:
from PIL import Image
import glob

class TestingDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.images = []
        self.names = []

        self.images = sorted(glob.glob(f"{self.img_dir}/*"))
        self.names = [os.path.basename(image)[:-4] for image in self.images]

    def __len__(self):
        return len(self.images)

    def __getnames__(self):
        return self.names

    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert("L")
        image = self.transform(image)
        return image, self.names[idx]

In [7]:
batch_size = 64
mu, st = 0, 255

In [8]:
test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.TenCrop(40),
    transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
    transforms.Lambda(lambda tensors: torch.stack([transforms.Normalize(mean=(mu,), std=(st,))(t) for t in tensors])),
])

In [9]:
test_data = TestingDataset(f"{unzip_path}/data/Images/test", test_transform)
print(f"Total testing data: {test_data.__len__()}")
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

Total testing data: 3589


In [10]:
# 載入 checkpoint
checkpoint = torch.load('/content/drive/MyDrive/ML/epoch_96')
# if using CPU:
# checkpoint = torch.load('/content/drive/MyDrive/ML/epoch_96', map_location=torch.device('cpu'))
net = Vgg().to(device)
net.load_state_dict(checkpoint["params"])

<All keys matched successfully>

In [11]:
import pandas as pd

def evaluate_and_save_csv(net, dataloader, index_mapping, output_csv_path):
    net = net.eval()
    predictions = []
    filenames = []  # 用於儲存檔案名稱

    # 建立索引到類別名稱的映射

    class_dic = {v: k for k, v in index_mapping.items()}
    reverse_index_mapping = {k: v for v, k in index_mapping.items()}

    with torch.no_grad():
        for images, filepaths in dataloader:
            images = images.to(device)

            bs, ncrops, c, h, w = images.shape
            images = images.view(-1, c, h, w)

            # 模型前向傳播
            outputs = net(images)
            # combine results across the crops
            outputs = outputs.view(bs, ncrops, -1)
            outputs = torch.sum(outputs, dim=1) / ncrops
            preds = outputs.argmax(dim=-1).cpu().numpy().tolist()

            # 添加檔案名稱和預測結果
            filenames.extend(filepaths)
            predictions.extend(preds)

    # 建立 DataFrame 並保存為 CSV
    submission = pd.DataFrame({"filename": filenames, "label": predictions})
    submission.to_csv(output_csv_path, index=False)
    print(f"Predictions saved to {output_csv_path}")

In [12]:
index_mapping = {"Angry": 0, "Disgust": 1, "Fear": 2, "Happy": 3, "Neutral": 4, "Sad": 5, "Surprise": 6}
output_csv_path = "/content/drive/MyDrive/ML/paper_VGG_96.csv"

evaluate_and_save_csv(net, test_loader, index_mapping, output_csv_path)

Predictions saved to /content/drive/MyDrive/ML/paper_VGG_96_test.csv
